In [5]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, CategoricalArrays
using Plots
using Distributions

In [8]:
const GRB_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-11


In [9]:
preferences_0 = CSV.read("preferences_0.csv", DataFrame);
preferences_1 = CSV.read("preferences_1.csv", DataFrame);

In [23]:
rho = 0.5;
# Create a new model

m = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables

#zij = 1 if i and j are matched, 0 otherwise
@variable(m, z[1:size(preferences_0,1), 1:size(preferences_1,1)], Bin)
@variable(m, ui[1:size(preferences_0,1)], Bin)
@variable(m, uj[1:size(preferences_1,1)], Bin)

# Counting variable for unmatched students

#t represents max of mi and mj
#@variable(m, t[1:size(preferences_0,1), 1:size(preferences_1,1)], Bin)

# Create objective

@objective(m, Min, sum(-z[i,j]*(preferences_0[i,j] + preferences_1[j, i]) for i in 1:size(preferences_0,1), j in 1:size(preferences_1,1)) + rho*(sum(ui) + sum(uj)))

# Create constraints

# Each person can only be matched to one person 
@constraint(m, [i in 1:size(preferences_0,1)], sum(z[i,j] for j in 1:size(preferences_1,1)) <= 1)
@constraint(m, [j in 1:size(preferences_1,1)], sum(z[i,j] for i in 1:size(preferences_0,1)) <= 1)

# If unmatched define ui and uj
@constraint(m, [i in 1:size(preferences_0,1)], ui[i] >= 1 - sum(z[i,j] for j in 1:size(preferences_1,1)))
@constraint(m, [j in 1:size(preferences_1,1)], uj[j] >= 1 - sum(z[i,j] for i in 1:size(preferences_0,1)))

optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 440 columns and 1640 nonzeros
Model fingerprint: 0x564d128a
Variable types: 0 continuous, 440 integer (440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -429.7094115
Presolve removed 0 rows and 92 columns
Presolve time: 0.01s
Presolved: 80 rows, 348 columns, 1272 nonzeros
Variable types: 0 continuous, 348 integer (348 binary)
Found heuristic solution: objective -430.5280884

Root relaxation: objective -4.799733e+02, 86 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    -479.9733377 -479.97334  0.00%

In [24]:
# Get value of zij

zij = value.(z)

20×20 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0   1.0   0.0
 -0.0  -0.0  -0.0   0.0   0.0   0.0     -0.0  -0.0   0.0   0.0  -0.0   0.0
 -0.0   1.0  -0.0  -0.0   0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0   0.0
 -0.0   0.0   0.0  -0.0  -0.0   1.0     -0.0   0.0  -0.0   0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0   0.0  -0.0     -0.0  -0.0  -0.0   0.0  -0.0   0.0
 -0.0  -0.0  -0.0   0.0  -0.0   0.0  …  -0.0   0.0  -0.0   0.0   0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0      0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0   0.0      0.0  -0.0  -0.0   1.0  -0.0  -0.0
 -0.0   0.0   0.0   0.0  -0.0  -0.0     -0.0   0.0   1.0   0.0  -0.0  -0.0
 -0.0   0.0  -0.0   0.0   1.0  -0.0      0.0  -0.0  -0.0   0.0   0.0  -0.0
 -0.0  -0.0  -0.0   0.0  -0.0  -0.0  …   0.0   0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   1.0   0.0  -0.0  -0.0      0.0   0.0   0.0   0.0   0.0  -0.0
 -0.0   0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -

In [22]:
show(stdout, "text/plain", sum(zij, dims=1))

1×20 Matrix{Float64}:
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0